In [3]:
%cd /data/majoroval/jupyter/RS-25/homework/week02

/data/majoroval/jupyter/RS-25/homework/week02


In [18]:
import zipfile
import requests

import numpy as np
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO
from textwrap import wrap
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import roc_auc_score, log_loss, ndcg_score

from framework import setup_logging

# EDA

In [4]:
train = pl.read_parquet('train.parquet')
test = pl.read_parquet('test.parquet')

# Если запускаетесь на кагле:
# train = train.sample(200000, shuffle=True)

train.head(100)

action_type,city_name,position_in_request,product_category,product_id,product_image,product_name,request_id,source_type,store_id,timestamp,user_id
str,str,i64,str,u64,str,str,u64,str,u64,i64,u64
"""AT_View""","""Москва""",36,"""ЛАПША ГРЕЧНЕВАЯ С МЯСОМ""",16123780718943738616,"""https://avatars.mds.yandex.net…","""Гречневая лапша с курицей, ово…",2648890305186553331,"""ST_Feed""",16009534250158248514,1679806419,9812600053596579429
"""AT_View""","""Краснодар""",4,"""Другие соленья""",9225194405337347900,"""https://avatars.mds.yandex.net…","""Грибы Опята маринованные «Гриб…",11772559297474986756,"""ST_Catalog""",17528705347053126993,1684047251,5490055591987012418
"""AT_View""","""Москва""",8,"""Другие виды вафель""",5093205019591406338,"""https://avatars.mds.yandex.net…","""Палочки вафельные «Коломенский…",6530124002700424792,"""ST_Search""",8036341040759991016,1687439633,7732070145991942829
"""AT_View""","""Санкт-Петербург""",6,"""Коласодержащие напитки с сахар…",6248957849722053876,"""https://avatars.mds.yandex.net…","""Напиток Cola «Добрый» газирова…",16719003150245349158,"""ST_OnClickDelivery""",13710539526554605580,1688540033,6666258464011831625
"""AT_View""","""Санкт-Петербург""",4,"""Зерновые чипсы""",1860732219874556850,"""https://avatars.mds.yandex.net…","""Чипсы цельнозерновые Корнерсы …",16169165339671437251,"""ST_Uplift""",12515600471032553786,1678793136,12902623331747310451
…,…,…,…,…,…,…,…,…,…,…,…
"""AT_View""","""Москва""",18,"""Мороженое в ведерках/лотках""",4962457401570396603,"""https://avatars.mds.yandex.net…","""Мороженое Monterra бельгийский…",3732239185086142913,"""ST_Catalog""",2253484422878426321,1687522515,4529653658662005692
"""AT_View""","""Москва""",7,"""СЫРНИКИ С НАЧИНКАМИ И ТОПИНГАМ…",18119989971542602749,"""https://avatars.mds.yandex.net…","""Сырники с вишней «Из Лавки», 1…",9213519401833166504,"""ST_Catalog""",15351512924493625964,1687415630,4376886766801706275
"""AT_Click""","""Москва""",null,"""Прочие замороженные торты""",10397281864504523812,"""https://avatars.mds.yandex.net…","""Торт Графские развалины «Из Ла…",14140071391418017592,"""ST_Search""",17885235751668191113,1697192345,9040343820851285402


In [5]:
def load_poster(row):
    title, poster_url = row[0], row[2]
    try:
        response = requests.get(poster_url, timeout=20)
        response.raise_for_status()
        return Image.open(BytesIO(response.content)), title

    except Exception as e:
        print(f"Error loading poster for '{title}': {e}")
        return None, None

def show_posters(data):
    fig, axes = plt.subplots(2, 5, figsize=(20, 10))
    plt.subplots_adjust(hspace=0.5, wspace=0.3)

    rows = [row for row in data.iter_rows()]

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(load_poster, rows))

    for idx, (img, title) in enumerate(results):
        ax = axes[idx//5, idx%5]
        if img and title:
            ax.imshow(img)
            wrapped_title = "\n".join(wrap(title, width=40))
            ax.set_title(wrapped_title)
        else:
            ax.set_title("Image not available", fontsize=10)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
class Experiment:
    def __init__(self, name, feature_sets, model, metrics):
        self.name = name
        self.feature_sets = feature_sets
        self.model = model
        self.metrics = metrics
        self.results = {}
        
    def run(self, train_fold, val_fold):
        """Run experiment on a single fold"""
        # Prepare features
        # Train model
        # Generate predictions
        # Calculate metrics
        
    def get_metrics(self, val_fold):
        

In [ ]:
class RankingMetrics:
    @staticmethod
    def map_at_k(true_relevance, predicted_scores, k=10):
        """Calculate MAP@K metric"""
        # Implementation
        
    @staticmethod
    def ndcg_at_k(true_relevance, predicted_scores, k=10):
        """Calculate NDCG@K metric"""
        # Implementation
        
    @staticmethod
    def novelty_at_k(recommendations, popularity_df, k=10):
        """Calculate Novelty@K metric"""
        # Implementation
        
    @staticmethod
    def serendipity_at_k(recommendations, user_history, k=10):
        """Calculate Serendipity@K metric"""
        # Implementation